In [1]:
import sys
from pathlib import Path
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
ROOT_PATH = Path().absolute().parent.parent
if str(ROOT_PATH) not in sys.path:
    sys.path.insert(1, str(ROOT_PATH))
from frequency_response import FrequencyResponse
from legacy_frequency_response import LegacyFrequencyResponse
from constants import DEFAULT_FS, PEQ_CONFIGS

In [2]:
def rmse(fr):
    ix10k = np.argmin(np.abs(10000 - fr.frequency))
    return np.sqrt(np.mean(np.square(fr.equalization[:ix10k] - fr.parametric_eq[:ix10k])))

In [3]:
def generate_stats(opt_fn, max_n=None):
    i = 0
    df_data = []
    for fp in ROOT_PATH.joinpath('results', 'oratory1990').glob('*/*/*.csv'):
        df_data.append(list(opt_fn(fp)))
        i += 1
        if max_n is not None and i >= max_n:
            break
    return pd.DataFrame(df_data, columns=['time', 'rmse', 'max_q'])

In [4]:
def opt_legacy55(fp):
    fr = LegacyFrequencyResponse.read_from_csv(fp)
    start_time = time()
    filter_matrix, _, _ = fr.optimize_parametric_eq([5, 5], DEFAULT_FS)
    return time() - start_time, rmse(fr), np.max(filter_matrix[:, 1])

In [5]:
def opt_legacy10(fp):
    fr = LegacyFrequencyResponse.read_from_csv(fp)
    start_time = time()
    filter_matrix, _, _ = fr.optimize_parametric_eq([10], DEFAULT_FS)
    return time() - start_time, rmse(fr), np.max(filter_matrix[:, 1])

In [6]:
def opt_neo4ls4hs(fp):
    fr = FrequencyResponse.read_from_csv(fp)
    configs = [PEQ_CONFIGS['4_PEAKING_WITH_LOW_SHELF'], PEQ_CONFIGS['4_PEAKING_WITH_HIGH_SHELF']]
    start_time = time()
    peqs = fr.optimize_parametric_eq(configs, fs=DEFAULT_FS)
    return time() - start_time, rmse(fr), np.max([np.max([filt.q for filt in peq.filters]) for peq in peqs])

In [7]:
def opt_neo8lshs(fp):
    fr = FrequencyResponse.read_from_csv(fp)
    configs = [PEQ_CONFIGS['8_PEAKING_WITH_SHELVES']]
    start_time = time()
    peqs = fr.optimize_parametric_eq(configs, fs=DEFAULT_FS)
    return time() - start_time, rmse(fr), np.max([np.max([filt.q for filt in peq.filters]) for peq in peqs])

### Old Optimizer with 5+5 Filters

In [8]:
generate_stats(opt_legacy55, max_n=None).describe()

,time,rmse,max_q
count,578.000000,578.000000,578.000000
mean,6.321416,1.335998,5.625031
std,1.177851,12.466280,1.949150
min,3.719745,0.077385,1.318174
25%,5.449458,0.285637,4.215849
50%,6.003759,0.371803,5.376187
75%,7.003599,0.487101,6.924262
max,10.819219,285.655209,12.922811


### New Optimizer with (4+1)+(4+1) Filters

In [10]:
generate_stats(opt_neo4ls4hs, max_n=None).describe()

,time,rmse,max_q
count,578.000000,578.000000,578.000000
mean,0.921271,0.422699,5.466687
std,0.244650,0.198339,0.896879
min,0.377085,0.092242,1.380708
25%,0.734165,0.274631,5.171088
50%,0.894701,0.389334,6.000000
75%,1.101747,0.530115,6.000000
max,1.638024,1.519409,6.000000


### New Optimizer with 8+2 Filters

In [11]:
generate_stats(opt_neo8lshs, max_n=None).describe()

,time,rmse,max_q
count,578.000000,578.000000,578.000000
mean,2.712405,0.317516,4.605426
std,0.298149,0.229024,1.226171
min,1.491334,0.030792,1.461956
25%,2.765620,0.172370,3.706195
50%,2.795627,0.259113,4.744047
75%,2.899400,0.390680,5.878069
max,3.146523,1.873839,6.000000


### Old Optimizer with 10 Filters

In [9]:
generate_stats(opt_legacy10, max_n=None).describe()

,time,rmse,max_q
count,578.000000,578.000000,578.000000
mean,3.264345,0.547724,2.930448
std,0.834060,0.251050,1.589851
min,2.255745,0.132055,0.399297
25%,2.693873,0.381384,1.717779
50%,2.992665,0.511192,2.606374
75%,3.595823,0.664518,3.800886
max,6.054808,3.212567,9.072648
